### Rerank

WPS process for the [rerank.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/rerank.R) from [ClimDown](https://github.com/pacificclimate/ClimDown)

**Quantile Reranking** fixes bias introduced by the [Climate Analogues (CA)](https://github.com/pacificclimate/ClimDown/blob/master/R/CA.R) step by re-applying a simple quantile mapping bias correction at each grid box

In [ ]:
from birdy import WPSClient
from netCDF4 import Dataset
from importlib.resources import files
from wps_tools.testing import get_target_url
from tempfile import NamedTemporaryFile
import requests
import os

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("chickadee")
print(f"Using chickadee on {url}")

Using chickadee on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/chickadee/wps


In [3]:
chickadee = WPSClient(url)

### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
chickadee.rerank?

Signature:
chickadee.rerank(
    obs_file,
    varname=None,
    qdm_file=None,
    analogues_object=None,
    num_cores='4',
    loglevel='INFO',
    analogues_name='analogues',
    units_bool=True,
    n_pr_bool=True,
    tasmax_units='celsius',
    tasmin_units='celsius',
    pr_units='kg m-2 d-1',
    max_gb=1.0,
    start_date=datetime.date(1971, 1, 1),
    end_date=datetime.date(2005, 12, 31),
    out_file=None,
    output_formats=None,
)
Docstring:
Quantile Reranking fixes bias introduced by the Climate Analogues step

Parameters
----------
obs_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of high-res gridded historical observations
varname : string
    Name of the NetCDF variable to downscale (e.g. 'tasmax')
out_file : string
    Filename to create with the climate imprint outputs
num_cores : {'1', '2', '3', '4'}positiveInteger
    The number of cores to use for parallel execution
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 

### Run the rerank process with RDA input

In [ ]:
with NamedTemporaryFile(suffix=".nc", prefix="output_", dir="/tmp", delete=True) as out_file:
    output = chickadee.rerank(
        qdm_file = (files("tests") / "data/QDM_expected_output.nc").resolve(),
        obs_file = (files("tests") / "data/tiny_obs.nc").resolve(),
        varname="tasmax",
        out_file=out_file.name,
        num_cores=2,
        analogues_object=(files("tests") / "data/analogues.rda").resolve()
    )
# Use asobj=True to access the output file contents as a dataset
output_dataset_rda = output.get(asobj=True)[0]

### Run the rerank process with RDS input

In [ ]:
with NamedTemporaryFile(suffix=".nc", prefix="output_", dir="/tmp", delete=False) as out_file:
    output = chickadee.rerank(
        qdm_file = (files("tests") / "data/QDM_expected_output.nc").resolve(),
        obs_file = (files("tests") / "data/tiny_obs.nc").resolve(),
        varname="tasmax",
        out_file=out_file.name,
        num_cores=2,
        analogues_object=(files("tests") / "data/analogues.rda").resolve(),
    )
output_dataset_rds = output.get(asobj=True)[0]

### Test for expected output

In [ ]:
expected_data = Dataset((files("tests") / "data/bccaq_expected_output.nc").resolve())
for key, value in expected_data.dimensions.items():
    assert str(output_dataset_rds.dimensions[key]) == str(value)
    assert str(output_dataset_rda.dimensions[key]) == str(value)